In [ ]:
import json

from climateforcing.utils import mkdir_p
import fair
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.stats as st

## Set up plotting styles

In [ ]:
pl.rcParams['figure.figsize'] = (12/2.54, 9/2.54)
pl.rcParams['font.size'] = 12
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['ytick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['ytick.right'] = True

## Load up saved output

In [ ]:
df_in = {}
for var in ['temperature', 'ERFaer']:
    df_in[var] = pd.read_csv('../data_output/fair_{}/ssp126.csv'.format(var), index_col='year')

In [ ]:
df_in['temperature']

In [ ]:
hadcrut5_df = pd.read_csv('../data_input/gsat/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv')

In [ ]:
mkdir_p('../plots/')

In [ ]:
n_ens = df_in['temperature'].shape[1]
n_ens

In [ ]:
fig,ax=pl.subplots()
ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'], axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750.5,2101), np.percentile(df_in['temperature'], 5, axis=1), np.percentile(df_in['temperature'], 95, axis=1), label='FaIR 5-95% range', color="0.7")
ax.plot(np.arange(1850.5,2022), hadcrut5_df.loc[:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1950,2100)
ax.set_ylim(0,2.5)
ax.axhline(1.5, ls=':', color='k')
ax.set_title('SSP1-2.6')
ax.legend(loc='lower right')
ax.set_ylabel('Temperature relative to 1850-1900, °C')
pl.figtext(1,0.02,'@chrisroadmap',ha='right', color='0.6')
pl.figtext(0,0.02,f'{n_ens} ensemble members, FaIRv{fair.__version__}')
#ax.text(0.99,0.01,'FaIRv1.6.2 2237 constrained ensemble members', transform=ax.transAxes, ha='right')
#ax.text(0.99,)
fig.tight_layout(rect=[0.0,0.05,1,1])
fig.patch.set_facecolor('white')
pl.savefig('../plots/ssp126-default.png')

## Characterise by aerosol forcing

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5].mean(axis=0)

In [ ]:
np.percentile(df_in['ERFaer'].loc[2005.5:2014.5].mean(axis=0), (5, 10, 50, 90, 95))

In [ ]:
print(np.mean(df_in['ERFaer'].loc[2005.5:2014.5].mean(axis=0)))
print(np.std(df_in['ERFaer'].loc[2005.5:2014.5].mean(axis=0)))

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=7324789)
aer_strong = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(df_in['ERFaer'].loc[2005.5:2014.5,str(i)].mean(axis=0), loc=-1.53, scale=0.10)/st.norm.pdf(-1.53, loc=-1.53, scale=0.10)
    if likelihood>=accept_prob[i]:
        aer_strong[i] = True
    #print(likelihood)
np.sum(aer_strong)

In [ ]:
pl.hist(df_in['ERFaer'].loc[2005.5:2014.5,aer_strong].mean(axis=0))

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_strong].mean().mean()

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_strong].mean().std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=83713790)
aer_weak = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(df_in['ERFaer'].loc[2005.5:2014.5,str(i)].mean(axis=0), loc=-0.44, scale=0.11)/st.norm.pdf(-0.44, loc=-0.44, scale=0.11)
    if likelihood>=accept_prob[i]:
        aer_weak[i] = True
    #print(likelihood)
np.sum(aer_weak)

In [ ]:
pl.hist(df_in['ERFaer'].loc[2005.5:2014.5,aer_weak].mean(axis=0))

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_weak].mean().mean()

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_weak].mean().std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=42417764)
aer_central = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(df_in['ERFaer'].loc[2005.5:2014.5,str(i)].mean(axis=0), loc=-0.99, scale=0.11)/st.norm.pdf(-0.99, loc=-0.99, scale=0.11)
    if likelihood>=accept_prob[i]:
        aer_central[i] = True
    #print(likelihood)
np.sum(aer_central)

In [ ]:
pl.hist(df_in['ERFaer'].loc[2005.5:2014.5,aer_central].mean(axis=0))

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_central].mean().mean()

In [ ]:
df_in['ERFaer'].loc[2005.5:2014.5,aer_central].mean().std()

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))
ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,aer_strong], axis=1), color='r')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,aer_strong], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,aer_strong], 95, axis=1), 
    color="r", alpha=0.2, 
    label='strong PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % 
    (df_in['ERFaer'].loc[2005.5:2014.5,aer_strong].mean().mean(), df_in['ERFaer'].loc[2005.5:2014.5,aer_strong].mean().std())
)

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,aer_central], axis=1), color='y')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,aer_central], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,aer_central], 95, axis=1), 
    color="y", alpha=0.2, label='moderate PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % 
    (df_in['ERFaer'].loc[2005.5:2014.5,aer_central].mean().mean(), df_in['ERFaer'].loc[2005.5:2014.5,aer_central].mean().std())
)

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,aer_weak], axis=1), color='b')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,aer_weak], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,aer_weak], 95, axis=1), 
    color="b", alpha=0.2, label='weak PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % 
    (df_in['ERFaer'].loc[2005.5:2014.5,aer_weak].mean().mean(), df_in['ERFaer'].loc[2005.5:2014.5,aer_weak].mean().std())
)
# ax.plot(np.arange(1995.5,2101), np.median(df_in['temperature'], axis=1), color='k')
# ax.fill_between(np.arange(1995.5,2101), np.percentile(df_in['temperature'], 5, axis=1), np.percentile(df_in['temperature'], 95, axis=1), label='AR6 ensemble (%.2f $\pm$ %.2f) W m$^{-2}$' % (df_in['ERFaer'].loc[2019.5,:].mean(), df_in['ERFaer'].loc[2019.5,:].std()), color="k", alpha=0.2)
ax.plot(
    np.arange(1850.5,2022), 
    hadcrut5_df.loc[:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), 
    color='k', label='HadCRUT5'
)
ax.set_xlim(1950,2100)
ax.set_ylim(0, 2.5)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_three_aer_forcings.png')

In [ ]:
aer_minus1_lb = df_in['ERFaer'].loc[2005.5:2014.5].mean()>-1

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,aer_minus1_lb], axis=1), color='b')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,aer_minus1_lb], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,aer_minus1_lb], 95, axis=1), 
    color="b", alpha=0.2, label='aerosol ERF lower bound -1 W m$^{-2}$'
)

ax.plot(
    np.arange(1850.5,2022), 
    hadcrut5_df.loc[:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), 
    color='k', label='HadCRUT5'
)
ax.set_xlim(1950,2100)
ax.set_ylim(0, 2.5)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_aer_lower_bound_-1.png')

## Characterise by ECS

In [ ]:
with open('../data_input/ar6-fair-samples/fair-1.6.2-wg3-params-slim-reconstrained.json') as f:
    config_list_variable = json.load(f)

In [ ]:
def get_ecs(cfg):
    return(cfg['F2x']/cfg['lambda_global'])

In [ ]:
ecs = np.zeros(n_ens)
for i, cfg in enumerate(config_list_variable):
    ecs[i] = get_ecs(cfg)

In [ ]:
pl.hist(ecs, bins=np.arange(0, 14, 0.5))
print(np.percentile(ecs, (5, 10, 16, 20, 50, 80, 84, 90, 95)))

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=31445701)
high_ecs = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(ecs[i], loc=4.6, scale=0.45)/st.norm.pdf(4.6, loc=4.6, scale=0.45)
    if likelihood>=accept_prob[i]:
        high_ecs[i] = True
    #print(likelihood)
np.sum(high_ecs)

In [ ]:
ecs[high_ecs].mean(), ecs[high_ecs].std()

In [ ]:
pl.hist(ecs[high_ecs])

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=43245452)
low_ecs = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(ecs[i], loc=1.98, scale=0.3)/st.norm.pdf(1.98, loc=1.98, scale=0.3)
    if likelihood>=accept_prob[i]:
        low_ecs[i] = True
    #print(likelihood)
np.sum(low_ecs)

In [ ]:
ecs[low_ecs].mean(), ecs[low_ecs].std()

In [ ]:
pl.hist(ecs[low_ecs])

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=424326843)
mid_ecs = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(ecs[i], loc=3, scale=0.31)/st.norm.pdf(3, loc=3, scale=0.31)
    if likelihood>=accept_prob[i]:
        mid_ecs[i] = True
    #print(likelihood)
np.sum(mid_ecs)

In [ ]:
ecs[mid_ecs].mean(), ecs[mid_ecs].std()

In [ ]:
pl.hist(ecs[mid_ecs])

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,high_ecs], axis=1), color='r')
ax.fill_between(np.arange(1750.5,2101), np.percentile(df_in['temperature'].loc[:,high_ecs], 5, axis=1), np.percentile(df_in['temperature'].loc[:,high_ecs], 95, axis=1), color="r", alpha=0.2, label='High ECS (%.2f $\pm$ %.2f) K' % (ecs[high_ecs].mean(), ecs[high_ecs].std()))
ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,mid_ecs], axis=1), color='y')
ax.fill_between(np.arange(1750.5,2101), np.percentile(df_in['temperature'].loc[:,mid_ecs], 5, axis=1), np.percentile(df_in['temperature'].loc[:,mid_ecs], 95, axis=1), color="y", alpha=0.2, label='Mid ECS (%.2f $\pm$ %.2f) K' % (ecs[mid_ecs].mean(), ecs[mid_ecs].std()))
ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,low_ecs], axis=1), color='b')
ax.fill_between(np.arange(1750.5,2101), np.percentile(df_in['temperature'].loc[:,low_ecs], 5, axis=1), np.percentile(df_in['temperature'].loc[:,low_ecs], 95, axis=1), color="b", alpha=0.2, label='Low ECS (%.2f $\pm$ %.2f) K' % (ecs[low_ecs].mean(), ecs[low_ecs].std()))
# ax.plot(np.arange(1995.5,2101), np.median(df_in['temperature'], axis=1), color='k')
# ax.fill_between(np.arange(1995.5,2101), np.percentile(df_in['temperature'], 5, axis=1), np.percentile(df_in['temperature'], 95, axis=1), label='AR6 ensemble (%.2f $\pm$ %.2f) K' % (ecs.mean(), ecs.std()), color="k", alpha=0.2)
ax.plot(np.arange(1850.5,2022), hadcrut5_df.loc[:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='k', label='HadCRUT5')
ax.set_xlim(1950,2100)
ax.set_ylim(0, 2.5)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_three_ecs_distributions.png')

In [ ]:
pl.hist(ecs[low_ecs], bins=np.arange(0,10,.4), alpha=0.4, density=True, color='b')
pl.hist(ecs[mid_ecs], bins=np.arange(0,10,.4), alpha=0.4, density=True, color='y')
pl.hist(ecs[high_ecs], bins=np.arange(0,10,.4), alpha=0.4, density=True, color='r')

## Characterise by rate of change of aerosol forcing

Do as a regression

### mean of 2010-2020 minus 2000-2010

In [ ]:
pl.hist(df_in['ERFaer'].loc[2010.5:2019.5].mean()-df_in['ERFaer'].loc[2000.5:2009.5].mean())
np.percentile(df_in['ERFaer'].loc[2010.5:2019.5].mean()-df_in['ERFaer'].loc[2000.5:2009.5].mean(), (5, 10, 50, 90, 95))

### regression slope

In [ ]:
rates = np.zeros(n_ens)
for i_ens in range(n_ens):
    rates[i_ens] = st.linregress(np.arange(2000.5, 2020), df_in['ERFaer'].loc[2000.5:2019.5, str(i_ens)]).slope

In [ ]:
pl.hist(rates)
pl.xlabel('W m$^{-2}$ yr$^{-1}$ 2000-2019')
np.percentile(rates, (5, 10, 50, 90, 95))

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=1293650)
recov_slow = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(rates[i], loc=0.003195, scale=0.001)/st.norm.pdf(0.003195, loc=0.003195, scale=0.001)
    if likelihood>=accept_prob[i]:
        recov_slow[i] = True
    #print(likelihood)
np.sum(recov_slow)

In [ ]:
pl.hist(rates[recov_slow])

In [ ]:
# 0.0035412 
rates[recov_slow].mean()

In [ ]:
rates[recov_slow].std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=40961094)
recov_fast = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(rates[i], loc=0.01259, scale=0.001)/st.norm.pdf(0.01259, loc=0.01259, scale=0.001)
    if likelihood>=accept_prob[i]:
        recov_fast[i] = True
    #print(likelihood)
np.sum(recov_fast)

In [ ]:
pl.hist(rates[recov_fast])

In [ ]:
# 0.01222895
rates[recov_fast].mean()

In [ ]:
rates[recov_fast].std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=n_ens, random_state=24678762)
recov_central = np.zeros(n_ens, dtype=bool)
for i in range(n_ens):
    likelihood = st.norm.pdf(rates[i], loc=0.007845, scale=0.001)/st.norm.pdf(0.007845, loc=0.007845, scale=0.001)
    if likelihood>=accept_prob[i]:
        recov_central[i] = True
    #print(likelihood)
np.sum(recov_central)

In [ ]:
pl.hist(rates[recov_central])

In [ ]:
# 0.00783677
rates[recov_central].mean()

In [ ]:
rates[recov_central].std()

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))
ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,recov_fast], axis=1), color='r')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,recov_fast], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,recov_fast], 95, axis=1), 
    color="r", alpha=0.2, 
    label='fast recovery (%.5f $\pm$ %.5f) W m$^{-2}$ yr$^{-1}$' % 
    (rates[recov_fast].mean(), rates[recov_fast].std())
)

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,recov_central], axis=1), color='y')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,recov_central], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,recov_central], 95, axis=1), 
    color="y", alpha=0.2, label='moderate recovery (%.5f $\pm$ %.5f) W m$^{-2}$ yr$^{-1}$' % 
    (rates[recov_central].mean(), rates[recov_central].std())
)

ax.plot(np.arange(1750.5,2101), np.median(df_in['temperature'].loc[:,recov_slow], axis=1), color='b')
ax.fill_between(
    np.arange(1750.5,2101), 
    np.percentile(df_in['temperature'].loc[:,recov_slow], 5, axis=1), 
    np.percentile(df_in['temperature'].loc[:,recov_slow], 95, axis=1), 
    color="b", alpha=0.2, label='slow recovery (%.5f $\pm$ %.5f) W m$^{-2}$ yr$^{-1}$' % 
    (rates[recov_slow].mean(), rates[recov_slow].std())
)
# ax.plot(np.arange(1995.5,2101), np.median(df_in['temperature'], axis=1), color='k')
# ax.fill_between(np.arange(1995.5,2101), np.percentile(df_in['temperature'], 5, axis=1), np.percentile(df_in['temperature'], 95, axis=1), label='AR6 ensemble (%.2f $\pm$ %.2f) W m$^{-2}$' % (df_in['ERFaer'].loc[2019.5,:].mean(), df_in['ERFaer'].loc[2019.5,:].std()), color="k", alpha=0.2)
ax.plot(
    np.arange(1850.5,2022), 
    hadcrut5_df.loc[:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), 
    color='k', label='HadCRUT5'
)
ax.set_xlim(1950,2100)
ax.set_ylim(0, 2.5)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_three_aer_recovery_rates.png')

## Make output data

In [ ]:
df_ecs = pd.DataFrame(
    {
        'ECS': ecs,
        'low_ECS': low_ecs,
        'mid_ECS': mid_ecs,
        'high_ECS': high_ecs
    }
)

In [ ]:
df_ecs

In [ ]:
mkdir_p('../data_output/')

In [ ]:
df_ecs.to_csv('../data_output/ecs.csv')

In [ ]:
df_aero_dists = pd.DataFrame(
    {
        'strong_aer': aer_strong,
        'mid_aer': aer_central,
        'weak_aer': aer_weak,
        'minus1_lb_aer': aer_minus1_lb
    }
)

In [ ]:
df_aero_dists.to_csv('../data_output/aerosol_distributions.csv')

## Show that the strong aerosol forcing is Higher 2C

Likelihood of 50-66% of staying under 2C for entire 21st century:
- peak median warming < 2C
- peak 66th percentile warming > 2C

In [ ]:
np.max(np.median(df_in['temperature'].loc[:,aer_strong], axis=1))

In [ ]:
np.max(np.percentile(df_in['temperature'].loc[:,aer_strong], 66, axis=1))

## Show that the weak aerosol forcing is Below 1.5C

Likelihood of 50-66% of staying under 1.5C for entire 21st century:
- peak median warming < 1.5C
- peak 66th percentile warming > 1.5C

there is a lower unused category of SR1.5 where the 66th percentile is less than 1.5C and in fact it falls into this too

In [ ]:
np.max(np.median(df_in['temperature'].loc[:,aer_weak], axis=1))

In [ ]:
np.max(np.percentile(df_in['temperature'].loc[:,aer_weak], 66, axis=1))